## Structure of the code flow:

#Step1: Import

#Step2: Env setup and Initialize

#Step3: Setup a Class to check the webpage

#Step4: Call GPT mini model

#Step5: Provide User and system prompts

#Step6: Create the brochure

#Step7: Stream the model outputs



In [12]:
#Import required libraries

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [13]:
# Env setup and Initialize

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key found and looks good so far!


In [14]:
# Functions to inititliaze, fetch requests, get contents (Instead of class website, functions are used here)

import requests
from bs4 import BeautifulSoup

class Website:
    """
    A simple utility class to represent a scraped website, including its title, text, and links.
    """

    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    def __init__(self, url):
        self.url = url
        self.title = "No title found"
        self.text = ""
        self.links = []

        self._fetch()

    def _fetch(self):
        response = requests.get(self.url, headers=self.HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')

        if soup.title:
            self.title = soup.title.string.strip()

        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)

        self.links = [link.get('href') for link in soup.find_all('a', href=True)]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n"



In [15]:
medium = Website("https://medium.com")
medium.links

['/',
 '/about?autoplay=1',
 '/membership',
 '/m/signin?operation=register&redirect=https%3A%2F%2Fmedium.com%2Fnew-story&source=--------------------------new_post_topnav------------------',
 '/m/signin?operation=login&redirect=https%3A%2F%2Fmedium.com%2F&source=--------------------------lo_home_nav------------------',
 '/m/signin?operation=register&redirect=https%3A%2F%2Fmedium.com%2F&source=--------------------------lo_home_nav------------------',
 '/about?autoplay=1',
 'https://help.medium.com/hc/en-us',
 'https://policy.medium.com/medium-terms-of-service-9db0094a1e0f',
 'https://policy.medium.com/medium-privacy-policy-f03bf92035c9',
 'https://help.medium.com/hc/en-us',
 'https://medium.statuspage.io',
 '/about?autoplay=1',
 '/jobs-at-medium/work-at-medium-959d1a85284e',
 'mailto:pressinquiries@medium.com',
 'https://blog.medium.com',
 'https://policy.medium.com/medium-privacy-policy-f03bf92035c9',
 'https://policy.medium.com/medium-rules-30e5502c4eb4',
 'https://policy.medium.com/me

In [16]:
## GPT 4o Mini to figure out which links in this website are relevant for the company brochure

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [17]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [18]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [19]:
print(get_links_user_prompt(medium))

Here is the list of links on the website of https://medium.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/about?autoplay=1
/membership
/m/signin?operation=register&redirect=https%3A%2F%2Fmedium.com%2Fnew-story&source=--------------------------new_post_topnav------------------
/m/signin?operation=login&redirect=https%3A%2F%2Fmedium.com%2F&source=--------------------------lo_home_nav------------------
/m/signin?operation=register&redirect=https%3A%2F%2Fmedium.com%2F&source=--------------------------lo_home_nav------------------
/about?autoplay=1
https://help.medium.com/hc/en-us
https://policy.medium.com/medium-terms-of-service-9db0094a1e0f
https://policy.medium.com/medium-privacy-policy-f03bf92035c9
https://help.medium.com/hc/en-us
https://medium.statuspage.io
/about?autoplay=1
/jobs-at-medium/work-at-

In [20]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [21]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/HiDream-ai/HiDream-I1-Full',
 '/microsoft/bitnet-b1.58-2B-4T',
 '/agentica-org/DeepCoder-14B-Preview',
 '/moonshotai/Kimi-VL-A3B-Thinking',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/HiDream-ai/HiDream-I1-Dev',
 '/spaces/Efficient-Large-Model/SanaSprint',
 '/spaces',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/openai/mrcr',
 '/datasets/zwhe99/DeepMath-103K',
 '/datasets/divaroffical/real_estate_ads',
 '/datasets/openai/graphwalks',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/W

In [22]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

### Create the Brochure


In [24]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [29]:
print(get_all_details("https://medium.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://medium.com/about?autoplay=1'}, {'type': 'careers page', 'url': 'https://medium.com/jobs-at-medium/work-at-medium-959d1a85284e'}, {'type': 'blog page', 'url': 'https://blog.medium.com'}]}
Landing page:
Webpage Title:
Medium: Read and write stories.

Webpage Contents:
Our story
Membership
Write
Sign in
Get started
Human
stories & ideas
A place to read, write, and deepen your understanding
Start reading
Start reading
About
Help
Terms
Privacy
Help
Status
About
Careers
Press
Blog
Privacy
Rules
Terms
Text to speech


about page
Webpage Title:
About Medium

Webpage Contents:
Sign in
Sign up
Everyone has a story to tell
Medium is a home for human stories and ideas. Here, anyone can share knowledge and wisdom with the world—without having to build a mailing list or a following first. The internet is noisy and chaotic; Medium is quiet yet full of insight. It’s simple, beautiful, collaborative, and helps you find the right readers for

In [30]:
## Setting up System Prompts

system_prompt = "You are an fun and cheerful assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
Respond cheerfully, politely in both English and Spanish"

In [31]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [32]:
get_brochure_user_prompt("Medium", "https://medium.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://medium.com/about?autoplay=1'}, {'type': 'careers page', 'url': 'https://medium.com/jobs-at-medium/work-at-medium-959d1a85284e'}, {'type': 'blog page', 'url': 'https://blog.medium.com'}]}


'You are looking at a company called: Medium\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nMedium: Read and write stories.\n\nWebpage Contents:\nOur story\nMembership\nWrite\nSign in\nGet started\nHuman\nstories & ideas\nA place to read, write, and deepen your understanding\nStart reading\nStart reading\nAbout\nHelp\nTerms\nPrivacy\nHelp\nStatus\nAbout\nCareers\nPress\nBlog\nPrivacy\nRules\nTerms\nText to speech\n\n\nabout page\nWebpage Title:\nAbout Medium\n\nWebpage Contents:\nSign in\nSign up\nEveryone has a story to tell\nMedium is a home for human stories and ideas. Here, anyone can share knowledge and wisdom with the world—without having to build a mailing list or a following first. The internet is noisy and chaotic; Medium is quiet yet full of insight. It’s simple, beautiful, collaborative, and helps you find the right readers for whatever you have to

In [33]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [35]:
create_brochure("Medium", "https://medium.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://medium.com/about?autoplay=1'}, {'type': 'careers page', 'url': 'https://medium.com/jobs-at-medium/work-at-medium-959d1a85284e'}, {'type': 'blog', 'url': 'https://blog.medium.com'}]}


# Welcome to Medium!

## 🌟 Your Home for Stories and Ideas 🌟

At Medium, **everyone has a story to tell.** Our platform is designed to help you **read, write, and deepen your understanding** of the world around you. With over 100 million people connecting each month—from software developers to amateur novelists—Medium is a beautiful, collaborative space for thoughtful conversation and deep insights.

---

## 🚀 Our Mission

We believe in the **power of words**. Our goal is to create a system that rewards thoughtful discussion and meaningful writing, distancing ourselves from the chaos of sensational journalism. We're dedicated to fostering a deeper understanding of the world through writing, making it a nourishing ground for ideas that inspire, empower, and provoke thought.

---

## 🌍 Who We Serve

At Medium, our members are the heartbeat of our community. With over a million subscribers, we provide a platform for individuals eager to share their knowledge and experiences, **free from ads and the exploitation of personal data**. Whether you're seeking inspiration or looking to express your own stories, Medium connects you with the right audience.

---

## 💼 Join Our Team

### Why Work at Medium?

We’re thrilled to foster a unique company culture focused on collaboration, creativity, and inclusivity. Here’s what we offer:

- **Meaningful Mission:** Work fueled by purpose and a vision for the future of online publishing.
  
- **Remote-First Culture:** Collaborate with colleagues from around the globe, supported by flexible working arrangements and perks like home office stipends.

- **A Creative Environment:** Engage in a culture of innovation and encouragement, from high-fives to sharing travel stories.

- **Generous Benefits:** Enjoy comprehensive health insurance, 401(k) plans, parental leave, and unlimited paid time off.

Interested in joining us? Check our [careers page](#) for open positions!

---

## ✏️ Start Your Medium Journey Today!

- **[Start Reading](#)** - Dive into a world filled with unique perspectives and engaging narratives.
  
- **[Start Writing](#)** - Share your own stories and ideas that matter to you.

- **[Become a Member](#)** - Support our mission while enriching your reading experience!

---

### **Let's Connect!**

Join us as we continue to enhance the world through thoughtful storytelling. 

---

## ¡Bienvenido a Medium!

### 🌟 Tu hogar para historias e ideas 🌟

En Medium, **cada uno tiene una historia que contar.** Nuestra plataforma está diseñada para ayudarte a **leer, escribir y profundizar tu comprensión** del mundo que te rodea. Con más de 100 millones de personas conectándose cada mes, desde desarrolladores de software hasta novelistas aficionados, Medium es un espacio hermoso y colaborativo para conversaciones reflexivas e ideas profundas.

---

### 🚀 Nuestra Misión

Creemos en el **poder de las palabras.** Nuestro objetivo es crear un sistema que recompense la discusión reflexiva y la escritura significativa, distanciándonos del caos del periodismo sensacionalista. Estamos dedicados a fomentar una comprensión más profunda del mundo a través de la escritura, haciendo de este un terreno fértil para ideas que inspiran, empoderan y provocan reflexión.


---

### 🌍 A Quien Servimos

En Medium, nuestros miembros son el corazón de nuestra comunidad. Con más de un millón de suscriptores, ofrecemos una plataforma para aquellos que desean compartir su conocimiento y experiencias, **libres de anuncios y la explotación de datos personales.** Ya sea que busques inspiración o quieras expresar tus propias historias, Medium te conecta con la audiencia adecuada.

---

### 💼 Únete a Nuestro Equipo

#### ¿Por Qué Trabajar en Medium?

Nos complace fomentar una cultura empresarial única centrada en la colaboración, la creatividad y la inclusividad. Esto es lo que ofrecemos:

- **Misión Significativa:** Trabaja impulsado por un propósito y una visión para el futuro de la publicación en línea.

- **Cultura Remota Primero:** Colabora con colegas de todo el mundo, soportado por opciones de trabajo flexibles y beneficios como estipendios para la oficina en casa.

- **Un Entorno Creativo:** Participa en una cultura de innovación y estímulo, desde chocar los cinco hasta compartir historias de viaje.

- **Beneficios Generosos:** Disfruta de un seguro de salud integral, planes 401(k), licencia parental y tiempo libre ilimitado.

¿Interesado en unirte a nosotros? Consulta nuestra [página de carreras](#) para las posiciones abiertas.

---

### ✏️ ¡Comienza Tu Viaje en Medium Hoy!

- **[Comienza a Leer](#)** - Sumérgete en un mundo lleno de perspectivas únicas y narrativas cautivadoras.
  
- **[Comienza a Escribir](#)** - Comparte tus propias historias e ideas que te importan.

- **[Conviértete en Miembro](#)** - Apoya nuestra misión mientras enriqueces tu experiencia de lectura.

---

### **¡Conectemos!**

Acompáñanos mientras continuamos mejorando el mundo a través de la narración reflexiva.

#  Improvement in the code to stream the response from AI Model.
In this context, stream is enabled to allow Open AI to send chunks of information as response and build the entire response iteratively so the user can see the response

In [36]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [37]:
stream_brochure("Medium", "https://medium.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://medium.com/about?autoplay=1'}, {'type': 'careers page', 'url': 'https://medium.com/jobs-at-medium/work-at-medium-959d1a85284e'}, {'type': 'blog', 'url': 'https://blog.medium.com'}]}


# Welcome to Medium!

**Read and write stories** with a community of over 100 million people who believe in the power of words! 🌟

---

## About Us

At Medium, we are more than just a platform to share stories; we are a vibrant home for **human ideas and insights**. Here’s what makes Medium special:

- **A Rich Community**: From software developers to amateur novelists, CEOs to passionate storytellers, Medium brings together diverse voices eager to share wisdom.
- **Depth Over Sensation**: In an era filled with noise, we promote thoughtful conversations, giving priority to meaningful content over shallow coverage.
- **Empowerment through Writing**: We believe that each person has a unique story to tell, and we strive to create a space where those stories can be shared freely without the need for a following.

---

## Join Our Membership

Become part of our growing community! 🌍

- **Connect** with other writers and readers.
- **Dive deeper** into topics that resonate with you and expand your horizons.
- **Share Your Insights**: Write your story and be heard.

---

## Careers at Medium

**Ready to make a difference?** 🚀

Join us in building the best place to read and write on the internet:

- **Meaningful Mission**: Everyone at Medium, affectionately known as "Medians," is driven by empathy, vision, and a commitment to a safer online environment.
- **Remote-First Culture**: Work from wherever you are! We support a flexible, remote-working environment complete with at-home benefits.
- **Creative Environment**: Engage in collaborative teamwork, participate in fun challenges, and express your creativity!
- **Generous Benefits**: Enjoy comprehensive health care, 401(k) plans, parental leave, and unlimited paid time off.

Explore career opportunities and join a team that values innovation and collaboration!

---

## Connect with Us

We’re excited to help you start your journey with Medium. Explore, read, and write! 

### Start Reading → [Medium](https://medium.com)
### Start Writing → [Medium](https://medium.com)
### Become a Member → [Medium](https://medium.com)

**Let’s make stories happen!**

---

## ¡Bienvenidos a Medium!

**Leer y escribir historias** con una comunidad de más de 100 millones de personas que creen en el poder de las palabras! 🌟

---

## Sobre Nosotros

En Medium, somos más que solo una plataforma para compartir historias; somos un hogar vibrante para **ideas e insights humanos**. Aquí está lo que hace a Medium especial:

- **Una comunidad rica**: Desde desarrolladores de software hasta novelistas, CEO hasta narradores apasionados, Medium reúne voces diversas ansiosas por compartir sabiduría.
- **Profundidad sobre Sensación**: En una era llena de ruido, promovemos conversaciones reflexivas, dando prioridad a contenido significativo sobre cobertura superficial.
- **Empoderamiento a través de la escritura**: Creemos que cada persona tiene una historia única que contar, y nos esforzamos por crear un espacio donde esas historias puedan ser compartidas libremente.

---

## Únete a Nuestra Membresía

¡Conviértete en parte de nuestra creciente comunidad! 🌍

- **Conéctate** con otros escritores y lectores.
- **Profundiza** en temas que resuenen contigo y expande tus horizontes.
- **Comparte tus Ideas**: Escribe tu historia y hazte escuchar.

---

## Carreras en Medium

**¿Listo para hacer una diferencia?** 🚀

Únete a nosotros en la construcción del mejor lugar para leer y escribir en internet:

- **Misión Significativa**: Todos en Medium, cariñosamente conocidos como "Medians", están impulsados por la empatía, la visión y un compromiso con un entorno en línea más seguro.
- **Cultura Remota**: Trabaja desde donde quieras. Apoyamos un ambiente de trabajo flexible y remoto completo con beneficios en casa.
- **Ambiente Creativo**: Involúcrate en trabajo en equipo colaborativo, participa en desafíos divertidos y expresa tu creatividad.
- **Beneficios Generosos**: Disfruta de atención médica integral, planes 401(k), licencia por maternidad/paternidad y tiempo libre ilimitado.

Explora las oportunidades laborales y únete a un equipo que valora la innovación y colaboración.

---

## Conéctate con Nosotros

Estamos emocionados de ayudarte a comenzar tu viaje con Medium. ¡Explora, lee y escribe!

### Comienza a Leer → [Medium](https://medium.com)
### Comienza a Escribir → [Medium](https://medium.com)
### Conviértete en Miembro → [Medium](https://medium.com)

**¡Hagamos que las historias sucedan!**

#  Challenge for this exercise is to add multishot prompting.... Doing below in the code
### For adding a spanish brochure -- its done and covered in the earlier code

In [ ]:
system_prompt = "You are an fun and cheerful assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
Respond cheerfully, politely in both English and Spanish"

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

#  Refined Version of Multi-shot Prompting

In [39]:
def stream_brochure(company_name, url):
    # Define previous interactions and system prompt for multi-shot prompting
    previous_interactions = [
        {"role": "system", "content": "You are a fun and cheerful assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors, and recruits. Respond in markdown.\
Include details of company culture, customers, and careers/jobs if you have the information.\
Respond cheerfully, politely in both English and Spanish."},
        
        # Example of a prior user request (can be expanded with more interactions)
        {"role": "user", "content": "Can you help me create a brochure for my company?"},  
        
        {"role": "assistant", "content": "Of course! I'd be happy to help you with that. Could you please provide the name of your company and its website?"},

        {"role": "user", "content": "My company is ABC Corp. Here's the website: https://www.abccorp.com."},

        {"role": "assistant", "content": "Got it! Thank you. Let me gather information from your website, and I'll create a brochure for ABC Corp."},

        # You can include more prior examples or questions here
    ]

    # Add the current user prompt as the next interaction
    previous_interactions.append(
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
    )

    # Generate the response with multi-shot prompting
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=previous_interactions,  # Include the entire conversation history
        stream=True  # Enable streaming to receive real-time responses
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Loop through the stream and build the response incrementally
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [40]:
stream_brochure("Medium", "https://medium.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://medium.com/about?autoplay=1'}, {'type': 'careers page', 'url': 'https://medium.com/jobs-at-medium/work-at-medium-959d1a85284e'}, {'type': 'blog', 'url': 'https://blog.medium.com'}]}


Sure thing! Here’s a cheerful brochure for Medium that highlights its mission, culture, customers, and career opportunities.


# 📚 Welcome to Medium: A Place for Human Stories!

### **Our Story**
At Medium, we believe **everyone has a story to tell!** We are a home for human stories and ideas, providing a platform where anyone can share their knowledge and wisdom with the world. Our mission is to **deepen our collective understanding** of the world through the power of writing. 

### **Why Choose Medium?**
- **A Thriving Community:** Join over **100 million people** who connect and share their wisdom on Medium every month! Our readers range from software developers to amateur novelists to CEOs—all eager to learn from each other!
- **Read & Write with Intention:** We foster a thoughtful space that values depth, nuance, and meaningful conversations, helping you discover ideas that resonate. 

### **Membership Matters!**
Unlike many platforms, Medium is **not ad-driven.** We are proud to be supported by a growing community of over **a million amazing members** who believe in our mission. Your membership helps keep our platform thriving!

### **Careers at Medium**
Joining Medium means becoming part of a passionate team dedicated to the art of storytelling! Here’s why you should consider a career with us:
- **Remote-First Culture:** Work from anywhere! Enjoy flexible work arrangements and a stipend for your home office expenses.
- **Creative Environment:** We’re all about innovation, collaboration, and fun! Participate in creative sessions, share ideas, and enjoy our vibrant culture.
- **Generous Benefits:** From comprehensive health insurance to **unlimited paid time off,** we’ve got you covered!

If you’re **thoughtful, empathetic**, and passionate about the future of online publishing, come work with us! Check out our open positions [here](https://www.abccorp.com/careers).

### **Join Us!**
Whether you’re ready to dive into reading, writing, or joining our team, **Medium is the place for you!** Let’s make the world a little more insightful, one story at a time.

---

In Spanish:
# 📚 ¡Bienvenido a Medium: Un Lugar para Historias Humanas!

### **Nuestra Historia**
En Medium, creemos que **todos tienen una historia que contar.** Somos un hogar para historias e ideas humanas, brindando una plataforma donde cualquiera puede compartir su conocimiento y sabiduría con el mundo. Nuestra misión es **profundizar nuestra comprensión colectiva** del mundo a través del poder de la escritura.

### **¿Por Qué Elegir Medium?**
- **Una Comunidad Vital:** ¡Únete a más de **100 millones de personas** que se conectan y comparten su sabiduría en Medium cada mes! Nuestros lectores van desde desarrolladores de software hasta novelistas aficionados y CEOs, ¡todos ansiosos por aprender unos de otros!
- **Lee y Escribe Con Intención:** Fomentamos un espacio reflexivo que valora la profundidad, la matiz y las conversaciones significativas, ayudándote a descubrir ideas que resuenan.

### **¡La Membresía Importa!**
A diferencia de muchas otras plataformas, Medium no se basa en anuncios. Estamos orgullosos de ser apoyados por una creciente comunidad de más de **un millón de sorprendentes miembros** que creen en nuestra misión. ¡Tu membresía ayuda a mantener nuestra plataforma viva!

### **Carreras en Medium**
Unirte a Medium significa formar parte de un equipo apasionado por el arte de contar historias. Aquí tienes razones para considerar una carrera con nosotros:
- **Cultura Remota:** ¡Trabaja desde cualquier lugar! Disfruta de horarios flexibles y un estipendio para tus gastos de oficina en casa.
- **Ambiente Creativo:** ¡Estamos en pro de la innovación, la colaboración y la diversión! Participa en sesiones creativas, comparte ideas y disfruta de nuestra cultura vibrante.
- **Beneficios Generosos:** Desde un seguro de salud integral hasta **licencias pagadas ilimitadas**, ¡te tenemos cubierto!

Si eres **reflexivo, empático** y apasionado por el futuro de la publicación en línea, ¡ven a trabajar con nosotros! Consulta nuestras posiciones abiertas [aquí](https://www.abccorp.com/careers).

### **¡Únete a Nosotros!**
Ya sea que estés listo para sumergirte en la lectura, escribir o unirte a nuestro equipo, **Medium es el lugar para ti.** ¡Hagamos del mundo un poco más perspicaz, una historia a la vez!


Feel free to modify any details as necessary! Hope this brochure serves you well! 🎉😊